In [3]:
import pandas as pd
import os
# from src.data_matching.main import get_edgelist

In [4]:
df_source=pd.read_csv("./tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv")
df_target=pd.read_csv("./tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv")
df_source.columns,df_target.columns

(Index(['musician', 'birthDate', 'familyNameLabel', 'givenNameLabel',
        'numberOfChildren', 'websiteLabel', 'residenceLabel', 'ethnicityLabel',
        'religionLabel', 'activityStart', 'twitterNameLabel', 'geniusNameLabel',
        'recordLabelLabel'],
       dtype='object'),
 Index(['musician', 'birthDate', 'familyNameLabel', 'givenNameLabel',
        'numberOfChildren', 'websiteLabel', 'residenceLabel', 'ethnicityLabel',
        'religionLabel', 'activityStart', 'twitterNameLabel', 'geniusNameLabel',
        'recordLabelLabel'],
       dtype='object'))

In [2]:
path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
# for file in os.listdir(path):
#     base_name=os.path.basename(file).split(".")[0]
#     path_file=os.path.join(path,file)
#     df=pd.read_csv(path_file)
#     reduced_df=df.sample(int(0.25*len(df)))
#     base_new_name=base_name+"_small.csv"
#     reduced_df.to_csv(os.path.join(path,base_new_name),index=False)


In [3]:
from src.data_matching.EmbDI.edgelist import EdgeList

def get_edgelist(input_file, out_dir, export: bool =True, overwrite: bool = True, dry_run: bool = False):
    """Translate an input CSV file into an edgelist."""
    dfpath = input_file
    base_name = os.path.basename(input_file).replace(".csv", ".txt")
    edgefile = os.path.join(out_dir, base_name)
    info_file = None
    df = pd.read_csv(dfpath, low_memory=False)
    pref = ["3#__tn", "3$__tt", "5$__idx", "1$__cid"]
    el = EdgeList(df, edgefile, pref, info_file, flatten=True)
    if dry_run:
        if export:
            el.convert_to_dict()
            gdict = el.convert_to_dict()
            print("el", el.convert_to_dict())
            print("gdict", gdict)
            g_nx = nx.from_dict_of_lists(gdict)
            n, _ = osp.splitext(edgefile)
            nx_fname = n + ".nx"
            pkl_fname = n + ".pkl"
            if overwrite:
                with open(nx_fname, "wb") as nx_file:
                    pickle.dump(g_nx, nx_file)
                with open(pkl_fname, "wb") as pkl_file:
                    pickle.dump(gdict, pkl_file)


In [4]:
input_path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
output_path="/mnt/c/Users/nkired/Documents/data-matching/tests/output_data"
for file in os.listdir(input_path):
    path_file=os.path.join(input_path,file)
    if "small" in file:
        get_edgelist(path_file, output_path)
        

340it [00:00, 2333.50it/s]


806it [00:00, 2619.71it/s]


1086it [00:00, 2388.34it/s]


750it [00:00, 1710.18it/s]


654it [00:00, 1765.01it/s]


...

13980it [00:04, 3064.97it/s]


1250it [00:00, 1528.78it/s]


10it [00:00, 948.14it/s]


5813it [00:04, 1354.35it/s]


2906it [00:02, 1390.19it/s]

In [68]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
# model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

In [69]:
from transformers import BartConfig, BartModel

# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig(d_model=16)

# Initializing a model (with random weights) from the facebook/bart-large style configuration
model = BartModel(configuration)

# Accessing the model configuration
configuration = model.config

In [194]:
import torch

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [222]:
att="an"("Nan")

AttributeError: 'str' object has no attribute 'contains'

In [223]:
from transformers import BartTokenizer, BartModel, BartConfig
# import torch.nn as nn
from transformers import BartConfig, BartModel

configuration = BartConfig(d_model=80)
model = BartModel(configuration)
configuration = model.config
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

inputs = tokenizer("my name is nour", return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
# last_hidden_states1 = outputs.last_hidden_state[0]
sentence_embeddings1 = mean_pooling(outputs, inputs['attention_mask'])[0]


inputs = tokenizer("my name is nour", return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state[0]
sentence_embeddings2 = mean_pooling(outputs, inputs['attention_mask'])[0]

In [224]:
cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
print(cos_sim(sentence_embeddings1.detach().numpy(), sentence_embeddings2.detach().numpy()))

0.8975244


In [113]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("HaotianQi/6-768-en-distilbert")
model = AutoModel.from_pretrained("HaotianQi/6-768-en-distilbert")

inputs = tokenizer("name is 9adour", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state


inputs = tokenizer("my patate is nour", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states1 = outputs.last_hidden_state


In [100]:
last_hidden_states1[0][0].detach().numpy()

array([ 0.22596829,  1.1337802 , -0.406336  , ...,  1.3532113 ,
        0.5006873 ,  0.04817429], dtype=float32)

In [104]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("HaotianQi/6-768-en-distilbert")
model = AutoModel.from_pretrained("HaotianQi/6-768-en-distilbert")

inputs = tokenizer("name is 9adour", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
last_hidden_states[0][0]

(…)lbert/resolve/main/tokenizer_config.json: 100%|██████████| 369/369 [00:00<00:00, 1.08MB/s]
(…)768-en-distilbert/resolve/main/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.63MB/s]
(…)n-distilbert/resolve/main/tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 3.84MB/s]
(…)8-en-distilbert/resolve/main/config.json: 100%|██████████| 657/657 [00:00<00:00, 689kB/s]
pytorch_model.bin: 100%|██████████| 268M/268M [00:09<00:00, 27.8MB/s] 


tensor([-2.9773e-01,  1.5724e-01, -1.2090e-01,  2.0131e-01, -5.4234e-03,
         1.1509e+00, -5.1944e-01, -5.4392e-01, -3.4535e-01, -5.3543e-01,
         4.5796e-01, -7.0982e-01, -9.2714e-01,  5.1401e-01, -5.1286e-02,
        -6.8508e-02, -6.6399e-01,  8.5144e-01,  6.8438e-02, -1.4652e+00,
         4.2446e-03,  1.1866e-01,  5.6276e-01,  1.9474e-01, -2.0074e-01,
         7.1826e-01,  6.5658e-01, -3.2080e-01,  4.7069e+00, -1.0368e+00,
         4.4640e-01,  5.2893e-01,  2.7580e-01, -1.3324e-01,  3.1890e-01,
         4.2724e-01,  1.1498e+00, -6.1332e-01,  2.7761e-01,  2.3686e+00,
         4.2171e-01,  4.6497e-01,  1.8477e-01,  1.3966e-01, -2.4073e-01,
         2.4145e-01, -1.1408e+00,  3.6580e-02, -7.1237e-01,  1.8648e-01,
         3.1811e-01,  2.5932e-01, -1.6287e-01,  7.1321e-02,  3.2491e-01,
        -8.0969e-01,  6.1864e-02, -1.4218e-01, -4.5625e-01,  3.7133e-01,
         4.2728e-01, -4.0076e-01,  8.7416e-02,  1.2392e-02, -1.7881e-01,
        -2.8869e-01, -1.9142e-01, -7.7388e-01, -2.3

In [2]:
import os 
walk_strategies = ['basic', 'node2vec', 'deepwalk', 'metapath2vec']
sentence_lengths = [20, 30, 50, 70,100]
n_sentences = [2000, 3000, 5000, 7000, 9000, 100000]
training_algorithms = ['word2vec', 'doc2vec']
learning_methods = ['skipgram', 'CBOW']
window_sizes = [2, 3, 4, 5]
dimensions = 64
print("output_dir",os.path.exists("./tests/slurm/scripts/"))
def generate_scripts(output_dir="./tests/slurm/scripts"):
    i=0
    for walk_strategy in walk_strategies:
        for sentence_length in sentence_lengths:
            for n_sentence in n_sentences:
                for training_algorithm in training_algorithms:
                    for learning_method in learning_methods:
                        for window_size in window_sizes:
                            if not os.path.exists(f"./tests/output_data/dataframes/musiciens_{walk_strategy}_{sentence_length}_{n_sentence}_{dimensions}_{window_size}_{training_algorithm}_{learning_method}.csv"):
                                # Crée un nom unique pour le script
                                script_name = f"script_job{i}.sh"
                                i+=1
                                # Génère le contenu du script Shell
                                script_content = f"""#!/bin/bash
        srun singularity exec /logiciels/containerCollections/CUDA11/pytorch-NGC-20-06-py3.sif $HOME/my_env/bin/python src/data_matching/main.py detect-similarity-all \\
        -i1 tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv \\
        -i2 tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv \\
        -nd 64 \\
        -ws {walk_strategy} \\
        -ns {n_sentence} \\
        -wl {sentence_length} \\
        -ta {training_algorithm} \\
        -lm {learning_method} \\
        -w {window_size}
        """

                                # Écrit le contenu du script Shell dans le fichier
                                with open(f"{output_dir}/{script_name}", "w") as script_file:
                                    script_file.write(script_content)

generate_scripts()


output_dir True


In [234]:
walk_strategies = ['basic', 'node2vec', 'deepwalk', 'metapath2vec']
sentence_lengths = [200, 300, 500, 9000]
n_sentences = [2000, 3000, 5000, 7000, 9000, 100000]
training_algorithms = ['word2vec', 'doc2vec']
learning_methods = ['skipgram', 'CBOW']
window_sizes = [2, 3, 4, 5]
dimensions = 64

In [1]:
from src.data_matching.main import detect_similarity_all
input_file1=
input_file2=
walk_strategy=
walk_length=
ndim=
window_size=
training_algorithm=
learning_method=
detect_similarity_all(input_file1, input_file2,walk_strategy,walk_length,ndim,window_size,training_algorithm,learning_method)

/home/nkired/.cache/pypoetry/virtualenvs/data-matching-1lo6pFQ1-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'input_file1' is not defined